In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch import optim
from torchsummary import summary
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from pthflops import count_ops


In [ ]:
training=datasets.FashionMNIST(root="data",train=True,download=True,transform=ToTensor())
testing=datasets.FashionMNIST(root="data",train=False,download=True,transform=ToTensor())
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

In [ ]:
train_dataloader=DataLoader(training,batch_size=64)
test_dataloader=DataLoader(testing,batch_size=64)

In [ ]:
from LeNet import *
        
    



In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model=NeuralNetwork().to(device)
learning_rate=0.001


In [ ]:
summary(model,input_size=(1,28,28))

In [ ]:
inp = torch.rand(1,1,28,28).to(device)
count_ops(model,inp)

In [ ]:
loss_fun=nn.CrossEntropyLoss()
optimiser=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [ ]:
def accuracy_check(loader,model):
    correct=0
    total=0
    model.eval()
    with torch.no_grad():
        for (x,y) in (loader):
            x=x.to(device=device)
            y=y.to(device=device)
            scores=model(x)
            scores=nn.functional.softmax(scores,dim=1)
            _,predictions=scores.max(1)
            correct+=(predictions==y).sum()
            total+= predictions.size(0)
    return float(correct/total)

In [ ]:
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img=img.mean(dim=0)
     # unnormalize
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg,cmap="gray")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [ ]:
def images_to_probs(net, images):
    output = model(images)
    # convert output probabilities to predicted class
    _, preds_tensor = torch.max(output, 1)
    preds = np.squeeze(preds_tensor.numpy())
    return preds, [nn.functional.softmax(el, dim=0)[i].item() for i, el in zip(preds, output)]


def plot_classes_preds(net, images, labels):
    preds, probs = images_to_probs(net, images)
    # plot the images in the batch, along with predicted and true labels
    fig = plt.figure(figsize=(12, 48))
    for idx in np.arange(4):
        ax = fig.add_subplot(1, 4, idx+1, xticks=[], yticks=[])
        matplotlib_imshow(images[idx], one_channel=True)
        ax.set_title("{0}, {1:.1f}%\n(label: {2})".format(
            classes[preds[idx]],
            probs[idx] * 100.0,
            classes[labels[idx]]),
                    color=("green" if preds[idx]==labels[idx].item() else "red"))
    return fig

In [ ]:
from torch.utils.tensorboard import SummaryWriter


In [ ]:
writer=SummaryWriter('tensor/LeNet-fashionMNIST')

In [15]:
running_loss = 0.0
for epoch in tqdm(range(100)):  # loop over the dataset multiple times

    for i, data in tqdm(enumerate(train_dataloader, 0)):
        inputs, labels = data
        optimiser.zero_grad()
        outputs = model(inputs)
        loss = loss_fun(outputs, labels)
        loss.backward()
        optimiser.step()

        running_loss += loss.item()
        if i % 100 == 99:    # every 100 mini-batches...

            # ...log the running loss
            writer.add_scalar('training loss',
                            running_loss / 1000,
                            epoch * len(train_dataloader) + i)
            writer.add_figure('predictions vs. actuals',
                            plot_classes_preds(model, inputs, labels),
                            global_step=epoch * len(train_dataloader) + i)
            writer.add_scalar("Accuracy",accuracy_check(test_dataloader,model),global_step=epoch * len(train_dataloader) + i)
            running_loss = 0.0
print('Finished Training')
        



938it [00:32, 28.93it/s]
938it [00:32, 28.96it/s][05:28<48:59, 32.66s/it]
 11%|█         | 11/100 [06:01<48:19, 32.58s/it]

In [ ]:
def accuracy_check(loader,model):
    correct=0
    total=0
    model.eval()
    with torch.no_grad():
        for (x,y) in (loader):
            x=x.to(device=device)
            y=y.to(device=device)
            scores=model(x)
            scores=nn.functional.softmax(scores,dim=1)
            _,predictions=scores.max(1)
            correct+=(predictions==y).sum()
            total+= predictions.size(0)
    print(float(correct/total))

In [ ]:
print(accuracy_check(train_dataloader,model))
print(accuracy_check(test_dataloader,model))

In [ ]:
EPOCH = 100
PATH = "model.pt"
LOSS = loss

torch.save({
            'epoch': EPOCH,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimiser.state_dict(),
            'loss': LOSS,
            }, PATH)